In [16]:
import requests
import json
import facebook
import re
from datetime import datetime
import concurrent.futures
from pymongo import MongoClient

In [17]:
token = input("your token")
client = MongoClient("mongodb://192.168.75.130:27017")
db = client["fb"]

In [18]:
def get_comments_of_comment(comment_id):
    comments = []
    obj_page = requests.get("https://graph.facebook.com/v2.6/%s?fields=comments&access_token=%s"%(comment_id,token)).json()["comments"]
    
    while True:
        for comment in obj_page["data"]:
            comment["created_time"] = datetime.strptime(comment["created_time"], "%Y-%m-%dT%H:%M:%S+0000")
            comment["comment_id"] = comment_id
            comments.append(comment)
            print("  finish {}".format(comment["id"]))


        if "next" in obj_page["paging"]:
            next_page = obj_page["paging"]["next"]
            obj_page = requests.get(next_page).json()
        else:
            break
        
    return comments

In [19]:
def main():
    collection_kp_comments = db["kp_comments"]
    collection_kp_comments_of_comment = db["kp_comments_of_comment"]
    for comment_id in collection_kp_comments.find({}, {"_id":0, "id": 1}):
        comments = get_comments_of_comment(comment_id["id"])
        collection_kp_comments_of_comment.insert_many(comments)

In [ ]:
if __name__ == "__main__":
    main()